In [2]:
2+3

5

In [4]:
from crewai.knowledge.source.base_knowledge_source import BaseKnowledgeSource
 
from typing import Dict, Any
from pydantic import Field
import requests

In [11]:
class WeatherKnowledgeSource(BaseKnowledgeSource):
    city: str = Field(description="City for which weather should be fetched")
    
    def load_content(self)->Dict[Any, str]:
        print(f"Fetching weather for {self.city}...")

        # Hardcoded coordinates for demo purposes (San Francisco)
        endpoint = "https://api.open-meteo.com/v1/forecast"
        params = {
                "latitude": 37.77,
                "longitude": -122.42,
                "current_weather": True
            }

        response = requests.get(endpoint, params=params)
        response.raise_for_status()
        weather_data = response.json().get("current_weather", {})
        formatted = self.format_weather(weather_data)
        return {self.city: formatted}
    def format_weather(self, data: dict) -> str:
        if not data:
            return "No weather data available."

        return (
            f"Current weather in {self.city}:\n"
            f"- Temperature: {data.get('temperature')}°C\n"
            f"- Wind Speed: {data.get('windspeed')} km/h\n"
            f"- Weather Code: {data.get('weathercode')}\n"
            f"- Time: {data.get('time')}"
        )
    def validate_content(self, content: Dict[Any, str]) -> bool:
        """
        Validate the content returned by load_content.
        Returns True if content is a dict with at least one non-empty string value.
        Adjust logic if BaseKnowledgeSource expects a different contract/signature.
        """
        if not isinstance(content, dict):
            return False
        if not content:
            return False
        for k, v in content.items():
            if not isinstance(v, str) or not v.strip():
                return False
        return True
        
    def add(self) -> None:
        content = self.load_content()
        for _, text in content.items():
            chunks = self._chunk_text(text)
            self.chunks.extend(chunks)

        self._save_documents()



In [16]:
from crewai import Agent, LLM

weather_knowledge = WeatherKnowledgeSource(city="San Francisco")

weather_agent = Agent(
    role="Weather Reporter",
    goal="Answer questions about the current weather forecast.",
    backstory="""You are a friendly meteorologist who
                  provides real-time weather updates.""",
    knowledge_sources=[weather_knowledge],
    llm=LLM(model="gpt-5-mini"),
    verbose=True
)


In [17]:
from crewai import Task, Crew, Process

task = Task(
    description="What is the current temperature and wind speed in SF?",
    expected_output="A concise weather summary for San Francisco.",
    agent=weather_agent
)

crew = Crew(
    agents=[weather_agent],
    tasks=[task],
    process=Process.sequential,
    verbose=True
)

In [ ]:
result = crew.kickoff()

print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8ac5b7aa-0601-41d1-a769-7f822c2c022e                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Fetching weather for San Francisco...


Output()

╭──────────────────────────────────────────── 📚 Retrieved Knowledge ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Additional Information: Current weather in San Francisco:                                                      │
│  - Temperature: 9.8°C                                                                                           │
│  - Wind Speed: 9.6 km/h                                                                                         │
│  - Weather Code: 0                                                                                              │
│  - Time: 2025-11-26T03:45                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Weather Reporter                                                                                        │
│                                                                                                                 │
│  Task: What is the current temperature and wind speed in SF?                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d8ec6690-ea8c-4137-a24d-8591c42972f5                                                                     │
│  Agent: Weather Reporter                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



┌───────────────────────── Tracing Preference Saved ──────────────────────────┐
│                                                                             │
│  Info: Tracing has been disabled.                                           │
│                                                                             │
│  Your preference has been saved. Future Crew/Flow executions will not       │
│  collect traces.                                                            │
│                                                                             │
│  To enable tracing later, do any one of these:                              │
│  • Set tracing=True in your Crew/Flow code                                  │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file              │
│  • Run: crewai traces enable                                                │
│                                                                             │
└─────────────────────────────────────

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 8ac5b7aa-0601-41d1-a769-7f822c2c022e                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Current weather in San Francisco (2025-11-26T03:45):                                             │
│  - Temperature: 9.8°C (≈49.6°F)                                                                                 │
│  - Wind speed: 9.6 km/h (≈6.0 mph)                                                                              │
│  - Weather code: 0 — Clear sky                                                                                  │
│                                                                                                                 │
│  Brief summary: It’s cool and clear with a light breeze. Would you like the hourly forecast or feels-like       │
│  temperature?                                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯